In [1]:
"""

EATING DISORDERS MONTECATINI

Purpose: HDDMrl model comparison for the three-groups data (HC, RI, AN).

Written by Corrado Caudek (2023-07-02)

"""

import datetime

now = datetime.datetime.now()
print("Current date and time : ")
print(now.strftime("%Y-%m-%d %H:%M:%S"))


Current date and time : 
2023-07-02 11:34:29


This notebook follows the [tutorial](https://hddm.readthedocs.io/en/latest/demo_RLHDDMtutorial.html#checking-results) on the hddm webpage.


In [2]:
%matplotlib inline 

import os, time, csv, sys
import glob

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

import pymc as pm
import hddm
import kabuki
import arviz as az

import pymc.progressbar as pbar
import pathlib

from kabuki.utils import concat_models
from kabuki.analyze import check_geweke
from kabuki.analyze import gelman_rubin

from patsy import dmatrix  # for generation of (regression) design matrices
import pickle

from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# Data management
pd.options.display.max_colwidth = 100

az.style.use("arviz-white")
%config InlineBackend.figure_format='retina'

print("The hddm version is", hddm.__version__)

/Users/corrado/mambaforge/envs/py37_env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The hddm version is 0.9.8


In [3]:
# Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Reset display options to their default values
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [4]:
!pwd

/Users/corrado/_repositories/eating_disorders_23/src/python/PRL/01_get_hddmrl_params


In [5]:
data = hddm.load_csv(
    "/Users/corrado/_repositories/eating_disorders_23/data/processed/prl/input_for_hddmrl/three_groups/ed_prl_data.csv"
)

In [6]:
print("There are %d participants" % data["subj_idx"].nunique())


There are 112 participants


In [7]:
data.head()

,subj_idx,response,stim,rt,trial,split_by,feedback,diag_cat,subj_code,q_init
0,1,0,food,0.979,1,0,0,AN,ca_po_2002_05_25_700_f,0.5
1,1,0,neutral,1.553,1,1,0,AN,ca_po_2002_05_25_700_f,0.5
2,1,1,food,1.939,2,0,0,AN,ca_po_2002_05_25_700_f,0.5
3,1,1,neutral,0.350,2,1,1,AN,ca_po_2002_05_25_700_f,0.5
4,1,0,food,0.768,3,0,0,AN,ca_po_2002_05_25_700_f,0.5


In [8]:
data.groupby("diag_cat")["subj_code"].nunique()

diag_cat
AN    36
HC    40
RI    36
Name: subj_code, dtype: int64

Check whether all AN patients have performance above 0.5 for at least one of the two conditions (food, neutral).

In [9]:
# Group the DataFrame and calculate the mean of 'feedback' for each combination of 'subj_code', 'stim', and 'diag_cat'
mean_feedback = data.groupby(["diag_cat", "stim"])["feedback"].mean()
print(mean_feedback)

diag_cat  stim   
AN        food       0.533088
          neutral    0.551705
HC        food       0.537656
          neutral    0.547969
RI        food       0.548214
          neutral    0.562305
Name: feedback, dtype: float64


In [10]:
data.columns

Index(['subj_idx', 'response', 'stim', 'rt', 'trial', 'split_by', 'feedback',
       'diag_cat', 'subj_code', 'q_init'],
      dtype='object')

In [11]:
# Calculate the minimum, maximum, and mean values of 'rt' for each combination of 'diag_cat' and 'stim'
result = data.groupby(["diag_cat", "stim"])["rt"].agg(["min", "max", "mean", "median"])
print(result)

                   min    max      mean  median
diag_cat stim                                  
AN       food     0.15  2.488  0.678686   0.550
         neutral  0.15  2.462  0.586113   0.480
HC       food     0.15  2.486  0.591276   0.453
         neutral  0.15  2.490  0.544739   0.422
RI       food     0.15  2.499  0.626885   0.473
         neutral  0.15  2.453  0.533328   0.412


## Model 1

In [12]:
m1 = hddm.HDDMrl(
    data,
    # bias=True,
    # depends_on={
    #     "a": ["diag_cat", "stim"],
    #     "v": ["diag_cat", "stim"],
    #     "t": ["diag_cat", "stim"],
    #     "alpha": ["diag_cat", "stim"],
    #     "pos_alpha": ["diag_cat", "stim"],
    # },
    # dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m1.find_starting_values()

m1.sample(2500, burn=500, dbname="models/ddm1.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm


/Users/corrado/mambaforge/envs/py37_env/lib/python3.7/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------100%-----------------] 2500 of 2500 complete in 976.5 sec

In [13]:
print("m1 DIC: %f" % m1.dic)

m1 DIC: 39879.443692


## Model 2

In [14]:
m2 = hddm.HDDMrl(
    data,
    # bias=True,
    # depends_on={
    #     "a": ["diag_cat", "stim"],
    #     "v": ["diag_cat", "stim"],
    #     "t": ["diag_cat", "stim"],
    #     "alpha": ["diag_cat", "stim"],
    #     "pos_alpha": ["diag_cat", "stim"],
    # },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m2.find_starting_values()

m2.sample(2500, burn=500, dbname="models/ddm2.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1237.7 sec

In [15]:
print("m2 DIC: %f" % m2.dic)

m2 DIC: 39124.890555


## Model 3

In [16]:
m3 = hddm.HDDMrl(
    data,
    # bias=True,
    depends_on={
        #     "a": ["diag_cat", "stim"],
        #     "v": ["diag_cat", "stim"],
        #     "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat"],
        "pos_alpha": ["diag_cat"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m3.find_starting_values()

m3.sample(2500, burn=500, dbname="models/ddm3.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1260.5 sec

In [17]:
print("m3 DIC: %f" % m3.dic)

m3 DIC: 39194.762673


## Model 4

In [18]:
m4 = hddm.HDDMrl(
    data,
    # bias=True,
    depends_on={
        #     "a": ["diag_cat", "stim"],
        #     "v": ["diag_cat", "stim"],
        #     "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat", "stim"],
        "pos_alpha": ["diag_cat", "stim"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m4.find_starting_values()

m4.sample(2500, burn=500, dbname="models/ddm4.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1781.9 sec

In [19]:
print("m4 DIC: %f" % m4.dic)

m4 DIC: 38197.466611


## Model 5

In [20]:
m5 = hddm.HDDMrl(
    data,
    # bias=True,
    depends_on={
        "a": ["diag_cat", "stim"],
        #     "v": ["diag_cat", "stim"],
        #     "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat", "stim"],
        "pos_alpha": ["diag_cat", "stim"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m5.find_starting_values()

m5.sample(2500, burn=500, dbname="models/ddm5.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1871.0 sec

In [21]:
print("m5 DIC: %f" % m5.dic)

m5 DIC: 36427.447802


## Model 6

In [22]:
m6 = hddm.HDDMrl(
    data,
    # bias=True,
    depends_on={
        "a": ["diag_cat", "stim"],
        "v": ["diag_cat", "stim"],
        #     "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat", "stim"],
        "pos_alpha": ["diag_cat", "stim"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m6.find_starting_values()

m6.sample(2500, burn=500, dbname="models/ddm6.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1866.8 sec

In [23]:
print("m6 DIC: %f" % m6.dic)

m6 DIC: 36185.146421


## Model 7

In [24]:
m7 = hddm.HDDMrl(
    data,
    # bias=True,
    depends_on={
        "a": ["diag_cat", "stim"],
        "v": ["diag_cat", "stim"],
        "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat", "stim"],
        "pos_alpha": ["diag_cat", "stim"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m7.find_starting_values()

m7.sample(2500, burn=500, dbname="models/ddm7.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 1779.3 sec

In [25]:
print("m7 DIC: %f" % m7.dic)

m7 DIC: 34904.053478


## Model 8

In [26]:
m8 = hddm.HDDMrl(
    data,
    bias=True,
    depends_on={
        "a": ["diag_cat", "stim"],
        "v": ["diag_cat", "stim"],
        "t": ["diag_cat", "stim"],
        "alpha": ["diag_cat", "stim"],
        "pos_alpha": ["diag_cat", "stim"],
    },
    dual=True,  # separate learning rates for pos/neg feedbacks
    p_outlier=0.05,
    informative=True,  # informative priors on ddm params
    include=["v", "a", "t"],
)

m8.find_starting_values()

m8.sample(2500, burn=500, dbname="models/ddm8.db", db="pickle")

No model attribute --> setting up standard HDDM
Set model to ddm
 [-----------------100%-----------------] 2501 of 2500 complete in 2116.0 sec

In [27]:
print("m8 DIC: %f" % m8.dic)

m8 DIC: 34917.762481
